In [68]:
from osgeo import gdal

In [69]:
dataset = gdal.Open(r"E:\Desktop\20221017-1022\20221020\2022-07-13_3_S2DL.tif")

In [70]:
description = dataset.GetDescription()  #获得栅格的描述信息
# 'E:\\Desktop\\20221017-1022\\20221020\\2022-07-13_3_S2DL.tif'

rcount = dataset.RasterCount  #获得栅格数据集的波段数
# 23
rX = dataset.RasterXSize  #栅格数据的宽度(X方向上的像素个数)
# 4659
rY = dataset.RasterYSize  #栅格数据的高度(Y方向上的像素个数)
#rY
extend = dataset.GetGeoTransform()  #栅格数据的六参数。
# (132.91275314469934,左上角经度坐标                           0
#  8.983152841195215e-05,X轴方向像元大小(水平空间分辨率)          1
#  0.0,X偏移 行旋转                                            2
#  47.43176565373803,左上角纬度坐标                             3
#  0.0,Y偏移  列旋转                                           4
# -8.983152841195215e-05)Y轴方向像元大小                        5


prj = dataset.GetProjection()  #栅格数据的投影
# 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],地理坐标系及参考椭球体
# PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],中央经线
# UNIT["degree",0.0174532925199433,指分辨率大小
# AUTHORITY["EPSG","9122"]],指的是EPSG数据集中UNIT为degree的ID是9122
# AXIS["Latitude",NORTH],
# AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'指的是EPSG数据集中坐标系ID为4326

In [71]:
bd = dataset.GetRasterBand(1)
# <osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x000001395CB85990> >

In [72]:
rs = bd.ReadAsArray()
rs

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

# 需求1 求栅格影像的四至坐标

In [73]:
extend = dataset.GetGeoTransform()
rX = dataset.RasterXSize
rY = dataset.RasterYSize

In [74]:
Xp = extend[0] + rX * extend[1] + rX * extend[2]
Yp = extend[3] + rY * extend[4] + rY * extend[5]
(Xp, Yp)

(133.33127823557064, 47.181045857940276)

In [75]:
(extend[0], extend[3])

(132.91275314469934, 47.43176565373803)

# 需求2 求SHP的四至坐标

In [76]:
from osgeo import ogr

datashp = ogr.Open(r'E:\Desktop\SHP\北京市范围.shp')
Ly = datashp.GetLayer(0)
dir(Ly)[:6] + ['... ...'] + dir(Ly)[-4:]

['AlterFieldDefn',
 'Clip',
 'CommitTransaction',
 'CreateFeature',
 'CreateField',
 'CreateFields',
 '... ...',
 '__weakref__',
 '_s',
 'schema',
 'this']

In [77]:
Ly.GetFeatureCount()

1

In [78]:
Ly.GetExtent()

(115.42341095999997, 117.51462503999996, 39.44275803, 41.06081600999998)

In [79]:
# Arcgis数据字段计算器计算四至坐标表达式
minX = !shape.extent.xmin!

maxX = !shape.extent.xmax!

minY = !shape.extent.ymin!

maxY = !shape.extent.ymax!

### 进阶需求 批量读取shp四至

In [88]:
import os

path = r'E:\Desktop\SHP'
files = os.listdir(path)
shp = []

In [89]:
for file in files:
    position = path + '\\' + file
    if ('.shp' in file) and ('.shp.' not in file):
        shp.append(position)

## ogr读取shp文件

In [90]:
import numpy as np
import pandas as pd

datapd = pd.DataFrame(columns=['xmin', 'xmax', 'ymin', 'ymax'])
for item in shp:
    datashp = ogr.Open(item)
    Ly = datashp.GetLayer(0)
    indexpd = item.split('\\')[-1]
    shpName = indexpd
    xmin = Ly.GetExtent()[0]
    xmax = Ly.GetExtent()[1]
    ymin = Ly.GetExtent()[2]
    ymax = Ly.GetExtent()[3]
    datapd.loc[shpName] = {'xmin': xmin, 'xmax': xmax, 'ymin': ymin, 'ymax': ymax}
    # Ly.GetExtent()

# frame = pd.read_excel('export_data/shpExtent.xlsx ', 'Sheet1')
writer = pd.ExcelWriter(r'./export_data/shpExtent.xlsx')  # 初始化一个writer
datapd.to_excel(writer, float_format='%.5f')  # table输出为excel, 传入writer
writer.save()  # 保存

### 进阶需求2 获取每个shp每个要素的四至范围